<a href="https://colab.research.google.com/github/abhi9rocks/DeepStuff/blob/master/TASK1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from IPython.display import HTML, clear_output
from subprocess import getoutput
s = getoutput('nvidia-smi')
if 'K80' in s:gpu = 'K80'
elif 'T4' in s:gpu = 'T4'
elif 'P100' in s:gpu = 'P100'
else:
    gpu='DONT PROCEED'
display(HTML(f"<h1>{gpu}</h1>"))

In [3]:
from google.colab import drive
from os import listdir, remove, makedirs
from os.path import join, exists
import random
drive.mount('/gdrive')
%cd /gdrive/MyDrive/darknet
!chmod +x ./darknet
!./darknet

Mounted at /gdrive
/gdrive/MyDrive/darknet
usage: ./darknet <function>


In [4]:
# change this path
inputpath="blazer/tops_fit"
if "gdrive" not in inputpath:
    inputpath = "/gdrive/MyDrive/innerwear/coat"
if exists(inputpath):
    x = input("Which model you want to train?\n\t1. tiny model\n\t2. tiny plus model\n\t3. big model\n")
    dont_run=False
    if x=='1':model_type = "tiny_model"
    elif x=='2':model_type = "tiny_plus_model"
    elif x=='3':model_type = "big_model"
    else:
        dont_run=True
        clear_output()
        print("Enter correct input among 1,2 and 3.")
else:
    print(inputpath, "cannot be found please correct it.")

Which model you want to train?
	1. tiny model
	2. tiny plus model
	3. big model
2


In [5]:
def check_existence(folder, count=0):
    tmp="" if count==0 else "_"+str(count)
    check_folder = folder+tmp
    if (check_folder in listdir(".")) and (len([k for k in listdir(check_folder) if ".weights" in k])!=0):
        count+=1
        return check_existence(folder, count)
    else:
        return check_folder
def get_all_files(folder:str, result:dict):
    tmp = sorted(listdir(folder))
    result[folder]=[i for i in tmp if (".jpg" in i) or (".txt" in i)]
    for i in tmp:
        if "." not in i:
            get_all_files(join(folder, i), result)
def get_images_txt(folder:str):
    data = {}
    get_all_files(folder, data)
    del data[folder]
    final_data = {}
    for i in data.keys():
        if len(data[i])==0:
            continue
        class_ = i[len(folder)+1:].split("/")[0]
        if class_ not in final_data.keys():
            final_data[class_]={}
        final_data[class_][i] = []
        for j in range(len(data[i])-1):
            if data[i][j][:data[i][j].rfind('.')]==data[i][j+1][:data[i][j+1].rfind('.')]:
                final_data[class_][i].append([i+'/'+data[i][j], i+'/'+data[i][j+1]])
                j+=1
    return final_data

In [6]:
if not dont_run:
    clear_output()
    outputpath="_".join(inputpath.split("/")[3:])+"_"+model_type
    outputpath = check_existence(outputpath)
    print("Output folder is "+outputpath+".")
    x = input("Please confirm to create the output folder (y/n):")
    if x == 'y':
        if not exists(outputpath):
            makedirs(outputpath)
        print("Output folder created.")
    elif x == 'n':
        print("Output folder not created.")
    else:
        print("Please input valid character.")

Output folder is innerwear_coat_tiny_plus_model.
Please confirm to create the output folder (y/n):y
Output folder created.


In [7]:
threshold = 10
data = get_images_txt(inputpath)
key_list = sorted(list(data.keys()))
count=0
for i in key_list:
    l = sum([len(k) for k in data[i].values()])
    if l>threshold:
        print(count,i, l)
        count+=1
    else:
        del data[i]
x = input("Please confirm that classes and count of images are correct (y/n):")
if x == 'y':
    classes = sorted(data.keys())
    print("Classes selected.")
elif x == 'n':
    classes = None
    print("Classes not selected.")
else:
    classes = None
    print("Please input valid character.")

0 dress 292
1 jumpsuit 195
2 shirts 269
3 tops 476
Please confirm that classes and count of images are correct (y/n):y
Classes selected.


In [8]:
for i in classes:
    print(i)
    for j in sorted(data[i].keys()):
        print(j, len(data[i][j]))
    print()

dress
/gdrive/MyDrive/innerwear/coat/dress/women 292

jumpsuit
/gdrive/MyDrive/innerwear/coat/jumpsuit/women 195

shirts
/gdrive/MyDrive/innerwear/coat/shirts/men 94
/gdrive/MyDrive/innerwear/coat/shirts/women 175

tops
/gdrive/MyDrive/innerwear/coat/tops/men 230
/gdrive/MyDrive/innerwear/coat/tops/women 246



In [9]:
for id in range(len(classes)):
    tmp = 0
    cls = classes[id]
    print(cls+"-", end = " ")
    for i in data[cls].keys():
        for j in data[cls][i]:
            f = open(j[1],'r')
            s = [k.split(" ") for k in f.read().split('\n')]
            s = [k for k in s if len(k)==5]
            s = [[str(id)]+k[1:] for k in s]
            s = "\n".join([" ".join(k) for k in s])+"\n"
            f.close()
            f = open(j[1],'w')
            f.write(s)
            f.close()
            tmp+=1
            if tmp%100==0:
                print(tmp, end=".. ")
    print(tmp)

dress- 100.. 200.. 292
jumpsuit- 100.. 195
shirts- 100.. 200.. 269
tops- 100.. 200.. 300.. 400.. 476


In [10]:
train=[]
for id in range(len(classes)):
    for i in data[classes[id]].keys():
        train.extend([k[0] for k in data[classes[id]][i]])
train=[i for i in train if ".txt" not in i]
random.shuffle(train)
l=len(train)
valid=train[int(l*5/6):]
train=train[:int(l*5/6)]
f=open(join(outputpath, model_type+"_train.txt"),'w')
f.write("\n".join(train))
f.close()
f=open(join(outputpath, model_type+"_valid.txt"),'w')
f.write("\n".join(valid))
f.close()

In [11]:
f=open(join(outputpath, model_type+"_obj.names"),'w')
f.write("\n".join(classes))
f.close()
numClass=len(classes)
objdata=["classes= "+str(numClass),
         "train = "+join(outputpath, model_type+"_train.txt"),
         "valid = "+join(outputpath, model_type+"_valid.txt"),
         "names = "+join(outputpath, model_type+"_obj.names"),
         "backup = "+outputpath+"/"]
f=open(join(outputpath, model_type+"_obj.data"),"w")
f.write("\n".join(objdata))
f.close()

In [12]:
import shutil
dest_cfg_name = model_type+"_yolov4.cfg"
if model_type == "tiny_model":
    src_cfg_name = "yolov4-tiny-custom.cfg"
    netSize=416
    batch=64
    subdiv=1
    max_batches_multiplier = 3000
    default_max_batch = 500200
elif model_type == "tiny_plus_model":
    src_cfg_name = "yolov4-tiny-custom.cfg"
    netSize=608
    batch=64
    subdiv=2
    max_batches_multiplier = 3000
    default_max_batch = 500200
elif model_type == "big_model":
    src_cfg_name = "yolov4-custom.cfg"
    netSize=416
    batch=64
    subdiv=1
    max_batches_multiplier = 2000
    default_max_batch = 500500
if dest_cfg_name not in listdir(outputpath):
    shutil.copy("cfg/"+src_cfg_name, outputpath+'/'+dest_cfg_name)
else:
    remove(outputpath+'/'+dest_cfg_name)
    shutil.copy("cfg/"+src_cfg_name, outputpath+'/'+dest_cfg_name)

In [16]:
#change number of class and filters
comm='s/classes=80/classes='+str(numClass)+"/"
!sed -i $comm $outputpath/$dest_cfg_name
comm='s/filters=255/filters='+str((numClass+5)*3)+"/"
!sed -i $comm $outputpath/$dest_cfg_name

#change number of steps and max batches
max_batches=max(6000,(numClass)*max_batches_multiplier)
comm='s/max_batches\ =\ '+str(default_max_batch)+'/max_batches\ =\ '+str(max_batches)+"/"
!sed -i $comm $outputpath/$dest_cfg_name
comm='s/steps=400000,450000/steps='+str(int(0.8*max_batches))+','+str(int(0.9*max_batches))+"/"
!sed -i $comm $outputpath/$dest_cfg_name

#can be multiple of 32 or 608(but makes training slow)
comm='s/width=416/width='+str(netSize)+"/"
!sed -i $comm $outputpath/$dest_cfg_name
comm='s/height=416/height='+str(netSize)+"/"
!sed -i $comm $outputpath/$dest_cfg_name

comm='s/batch=64/batch='+str(batch)+'/'
!sed -i $comm $outputpath/$dest_cfg_name
comm='s/subdivisions=1/subdivisions='+str(subdiv)+'/'
!sed -i $comm $outputpath/$dest_cfg_name

In [17]:
if "yolov4.conv.137" not in listdir():
    !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137
if "yolov4-tiny.conv.29" not in listdir():
    !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

In [ ]:
obj_file = None
cfg_file = None

if "tiny_model" in outputpath : model_name = "yolov4-tiny.conv.29"
elif "tiny_plus_model" in outputpath : model_name = "yolov4-tiny.conv.29"
elif "big_model" in outputpath : model_name = "yolov4.conv.137"

for i in listdir(outputpath):
    if "obj.data" in i:
        obj_file = join(outputpath, i)
    elif ".cfg" in i:
        cfg_file = join(outputpath, i)

if obj_file == None:
    print(".data file not found in", outputpath)

if cfg_file ==None:
    print(".cfg file not found in", outputpath)

if (obj_file != None) and (cfg_file != None):
    !./darknet detector train $obj_file $cfg_file $model_name -dont_show -map

Streaming output truncated to the last 5000 lines.
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 374138, rewritten_bbox = 0.000000 % 

 (next mAP calculation at 5600 iterations) 

 Tensor Cores are used.
 Last accuracy mAP@0.5 = 64.81 %, best = 67.45 % 
 5579: 0.257108, 0.214977 avg loss, 0.002610 rate, 1.652482 seconds, 357056 images, 3.067812 hours left
Loaded: 0.092505 seconds
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.847864), count: 33, class_loss = 0.312200, iou_loss = 0.166051, total_loss = 0.478251 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 374171, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.843789), co

In [4]:
outputpath = input("Write the outputpath where previous training was done?\n")


obj_file = None
cfg_file = None
last_weight = None
folder_exists = outputpath
if folder_exists:
    for i in outputpath:
        if "obj.data" in i:
            obj_file = join(outputpath, i)
        elif ".cfg" in i:
            cfg_file = join(outputpath, i)
        elif "last.weights" in i:
            last_weight = join(outputpath, i)
else:
    print(outputpath, "does not exist please check the name.")

if (obj_file == None) and (folder_exists):
    print(".data file not found in", outputpath)
if (cfg_file ==None) and (folder_exists):
    print(".cfg file not found in", outputpath)
if (last_weight ==None) and (folder_exists):
    print("last.weights file not found in", outputpath)

if (obj_file != None) and (cfg_file != None) and (last_weight != None):
    !./darknet detector train $obj_file $cfg_file $last_weight -dont_show -map

Write the outputpath where previous training was done?
innerwear_coat_tiny_plus_model
.data file not found in innerwear_coat_tiny_plus_model
.cfg file not found in innerwear_coat_tiny_plus_model
last.weights file not found in innerwear_coat_tiny_plus_model
